In [ ]:
import subprocess
import json
from pathlib import Path
from collections import defaultdict

from build_tools import run_all_checks, rebuild_tox_environments

PROJECT_DIR = Path.cwd().parent
SECURITY_DIR = Path.cwd() / "security"
SECURITY_DIR.mkdir(exist_ok=True)

print(f"Project: {PROJECT_DIR}")

## Quality Checks

In [ ]:
run_all_checks(PROJECT_DIR)

## Generate Third-Party Licenses

In [ ]:
pip_licenses = PROJECT_DIR / "venv" / "Scripts" / "pip-licenses.exe"

if not pip_licenses.exists():
    print(f"[WARNING] pip-licenses not found. Install with: pip install pip-licenses")
else:
    result = subprocess.run(
        [str(pip_licenses), "--from=mixed", "--format=markdown", "--with-urls",
         "--with-description", "--ignore-packages", "snowlib"],
        capture_output=True, text=True
    )

    if result.returncode == 0:
        licenses_dir = PROJECT_DIR / "licenses"
        licenses_dir.mkdir(exist_ok=True)
        with open(licenses_dir / "THIRD_PARTY.md", "w", encoding="utf-8") as f:
            f.write(result.stdout)
        print(f"[OK] Generated licenses/THIRD_PARTY.md")
    else:
        print(f"[ERROR] {result.stderr}")

## Tox: Test Across Python Versions

In [ ]:
def run_tox(rebuild: bool = False, envlist: list[str] | None = None):
    """Run tox tests"""
    stdout, stderr, returncode = rebuild_tox_environments(
        PROJECT_DIR, rebuild=rebuild, envlist=envlist
    )

    # Save log
    log_file = Path.cwd() / "tox_output.log"
    with open(log_file, "w", encoding="utf-8") as f:
        f.write(stdout)
        if stderr:
            f.write(f"\n\nSTDERR:\n{stderr}")

    print(stdout)
    if stderr:
        print(f"STDERR: {stderr}")

    print("\n" + "=" * 60)
    if returncode == 0:
        print("[OK] All tests passed!")
    else:
        print(f"[FAILED] Exit code {returncode}")
    print(f"Log saved to: {log_file}")

In [ ]:
# Run tox (set rebuild=True to recreate environments)
run_tox(rebuild=False, envlist=None)

## Security Scans

In [ ]:
# Bandit - Python security scanner
try:
    result = subprocess.run(
        ["bandit", "-r", str(PROJECT_DIR / "src"), "-f", "txt"],
        capture_output=True, text=True, encoding="utf-8", errors="replace"
    )

    with open(SECURITY_DIR / "bandit.log", "w", encoding="utf-8") as f:
        f.write(result.stdout)

    print(result.stdout)
    print("\n" + "=" * 60)
    if result.returncode == 0:
        print("[OK] Bandit: No security issues")
    else:
        print("[WARNING] Bandit: Issues detected")
except FileNotFoundError:
    print("[SKIP] Bandit not installed. Install with: pip install bandit")

In [ ]:
# Detect-secrets - Secret scanner
try:
    result = subprocess.run(
        ["detect-secrets", "scan", str(PROJECT_DIR / "src")],
        capture_output=True, text=True, encoding="utf-8", errors="replace"
    )

    with open(SECURITY_DIR / "detect-secrets.log", "w", encoding="utf-8") as f:
        f.write(result.stdout)

    try:
        scan = json.loads(result.stdout)
        secrets_count = sum(len(s) for s in scan.get("results", {}).values())
        if secrets_count == 0:
            print("[OK] No secrets detected")
        else:
            print(f"[WARNING] Found {secrets_count} potential secret(s)")
            for path, secrets in scan.get("results", {}).items():
                for s in secrets:
                    print(f"  {path}:{s.get('line_number')} - {s.get('type')}")
    except json.JSONDecodeError:
        print(result.stdout)
except FileNotFoundError:
    print("[SKIP] detect-secrets not installed. Install with: pip install detect-secrets")